In [1]:
# Load data from training data directory, convert from NPZ to tensor, and save to a dict
# containing lift name and tensor data

import os
import torch
import numpy as np
import functions

DEADLIFT_TENSORS = {}
BENCH_TENSORS = {}
SQUAT_TENSORS = {}

# Setup paths
script_dir = os.path.dirname(os.path.abspath("model_train.ipynb"))
data_dir = os.path.normpath(os.path.join(script_dir, '..', 'training data'))

# Step 1: Get all subfolder names in data_dir
subfolders = []
for f in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, f)):
        subfolders.append(f)


#  Squat Files
#  Bench Files
#  Deadlift Files
for subfolder in subfolders:
    subfolder_path = os.path.join(data_dir, subfolder)
    files = []

    # Isolate individual lift files in each of the subfolders and add them to files list
    for f in os.listdir(subfolder_path):
        if os.path.isfile(os.path.join(subfolder_path, f)):
            files.append(f)

    # Iterate through files in each subfolder
    for file in files:
        data = np.load(os.path.join(subfolder_path, file))

        # convert each landmark/angle to a tensor and add it to the tensor dict (which is
        # specific to the current lift only
        tensor_dict = {}
        for key in data.files:
            lift_name_without_video_type = " ".join(key.split('.')[:1] + key.split("_")[-2:])
            tensor_dict[lift_name_without_video_type] = torch.tensor(data[key])
        if file.split('.')[0].startswith('bench'):
            BENCH_TENSORS[file.split('.')[0]] = tensor_dict
        if file.split('.')[0].startswith('deadlift'):
            DEADLIFT_TENSORS[file.split('.')[0]] = tensor_dict
        if file.split('.')[0].startswith('squat'):
            SQUAT_TENSORS[file.split('.')[0]] = tensor_dict

In [2]:
BENCH_TENSORS['bench 2 good lift data'].keys()

dict_keys(['bench 2 good landmark 0', 'bench 2 good landmark 1', 'bench 2 good landmark 2', 'bench 2 good landmark 3', 'bench 2 good landmark 4', 'bench 2 good landmark 5', 'bench 2 good landmark 6', 'bench 2 good landmark 7', 'bench 2 good landmark 8', 'bench 2 good landmark 9', 'bench 2 good landmark 10', 'bench 2 good landmark 11', 'bench 2 good landmark 12', 'bench 2 good landmark 13', 'bench 2 good landmark 14', 'bench 2 good landmark 15', 'bench 2 good landmark 16', 'bench 2 good landmark 17', 'bench 2 good landmark 18', 'bench 2 good landmark 19', 'bench 2 good landmark 20', 'bench 2 good landmark 21', 'bench 2 good landmark 22', 'bench 2 good landmark 23', 'bench 2 good landmark 24', 'bench 2 good landmark 25', 'bench 2 good landmark 26', 'bench 2 good landmark 27', 'bench 2 good landmark 28', 'bench 2 good landmark 29', 'bench 2 good landmark 30', 'bench 2 good landmark 31', 'bench 2 good landmark 32', 'bench 2 good left elbow', 'bench 2 good right elbow', 'bench 2 good left k

In [3]:
for k,v in BENCH_TENSORS.items():
    print(k)
    for key in v.keys():

        try:
            print(f"---{key} contains a {type(v[key])} containing Z coords for {functions.landmark_indeces_to_labels[int(key.split('_')[-1])]} that contains {len(v[key])} points")
        except:
            print(f"---{key} contains a {type(v[key])} containing joint angles for {' '.join(key.split('_')[-2:])} that contains ({len(v[key])} angles)")

bench 2 good lift data
---bench 2 good landmark 0 contains a <class 'torch.Tensor'> containing joint angles for bench 2 good landmark 0 that contains (3 angles)
---bench 2 good landmark 1 contains a <class 'torch.Tensor'> containing joint angles for bench 2 good landmark 1 that contains (3 angles)
---bench 2 good landmark 2 contains a <class 'torch.Tensor'> containing joint angles for bench 2 good landmark 2 that contains (3 angles)
---bench 2 good landmark 3 contains a <class 'torch.Tensor'> containing joint angles for bench 2 good landmark 3 that contains (3 angles)
---bench 2 good landmark 4 contains a <class 'torch.Tensor'> containing joint angles for bench 2 good landmark 4 that contains (3 angles)
---bench 2 good landmark 5 contains a <class 'torch.Tensor'> containing joint angles for bench 2 good landmark 5 that contains (3 angles)
---bench 2 good landmark 6 contains a <class 'torch.Tensor'> containing joint angles for bench 2 good landmark 6 that contains (3 angles)
---bench 2 

In [4]:
# for k, v in BENCH_TENSORS.items():
#     for lift, coordinate in v.items():
#         print(lift, "coords",coordinate[2])

for key, value in DEADLIFT_TENSORS.items():
    print(key)
    # for landmark, coordinate in BENCH_TENSORS[key].items():
    #     return (landmark, coordinate[2])

deadlift 1 good lift data
deadlift 2 good_rotate_15 lift data
deadlift 3 bad lift data
deadlift 4 bad lift data


In [5]:
# Isolate z coordinates of individual lift for each lift category
# It is important to have them isolated as the model will have to train each lift individually
# LOGIC WILL BE IMPROVED LATER TO MAKE IT MORE DYNAMIC
#
# Stores as a dict of dicts

deadlift_z_coordinates = {}
for k,v in DEADLIFT_TENSORS.items():
    for lift, landmarks in v.items():
        if len(landmarks) == 3:
            lift_z_coords = f"{' '.join(lift.split(' ')[:3])} {functions.landmark_indeces_to_labels[int(' '.join(lift.split(' ')[-1:]))]} z coordinates"
            deadlift_z_coordinates[lift_z_coords] = landmarks[2]


bench_z_coordinates = {}
for k,v in BENCH_TENSORS.items():
    for lift, landmarks in v.items():
        if len(landmarks) == 3:
            lift_z_coords = f"{' '.join(lift.split(' ')[:3])} {functions.landmark_indeces_to_labels[int(' '.join(lift.split(' ')[-1:]))]} z coordinates"
            bench_z_coordinates[lift_z_coords] = landmarks[2]

squat_z_coordinates = {}
for k,v in SQUAT_TENSORS.items():
    for lift, landmarks in v.items():
        if len(landmarks) == 3:
            lift_z_coords = f"{' '.join(lift.split(' ')[:3])} {functions.landmark_indeces_to_labels[int(' '.join(lift.split(' ')[-1:]))]} z coordinates"
            squat_z_coordinates[lift_z_coords] = landmarks[2]
# Example of how to isolate one landmark in a single lift
print(squat_z_coordinates.keys())


dict_keys(['squat 1 good nose z coordinates', 'squat 1 good left eye inner z coordinates', 'squat 1 good left eye z coordinates', 'squat 1 good left eye outer z coordinates', 'squat 1 good right eye inner z coordinates', 'squat 1 good right eye z coordinates', 'squat 1 good right eye outer z coordinates', 'squat 1 good left ear z coordinates', 'squat 1 good right ear z coordinates', 'squat 1 good mouth left z coordinates', 'squat 1 good mouth right z coordinates', 'squat 1 good left shoulder z coordinates', 'squat 1 good right shoulder z coordinates', 'squat 1 good left elbow z coordinates', 'squat 1 good right elbow z coordinates', 'squat 1 good left wrist z coordinates', 'squat 1 good right wrist z coordinates', 'squat 1 good left pinky z coordinates', 'squat 1 good right pinky z coordinates', 'squat 1 good left index z coordinates', 'squat 1 good right index z coordinates', 'squat 1 good left thumb z coordinates', 'squat 1 good right thumb z coordinates', 'squat 1 good left hip z co

In [10]:
# Calling Function to determine which side of the lift we are viewing from
side = functions.determineSide('deadlift 4 bad', deadlift_z_coordinates)
side

The lower the z coordinate, the closer to the camera
Mean of the mean left side landmark z distance: 0.12165702829824966
Mean of the mean right side landmark z distance: -0.13286598812553013


'viewing from right'

In [27]:
from torch.utils.data import Dataset
import numpy as np

class DeadliftDataset(torch.utils.data.Dataset):
    def __init__(self, lift_tensor_dict, window_size=60, stride=10):
        self.samples = []
        self.window_size = window_size

        for lift_name, lift_data in lift_tensor_dict.items():
            # Get keys that are NOT landmarks
            print(lift_name, lift_data)
            angle_keys = [k for k in lift_data if "landmark" not in k.name]
            if len(angle_keys) != 8:
                print(f"Skipping {lift_name} — expected 8 joints, got {len(angle_keys)}")
                continue

            # Sort keys for consistency
            angle_keys.sort()

            # Stack each joint angle to form [num_frames, 8]
            joint_matrix = np.stack([lift_data[k] for k in angle_keys], axis=1)

            # Create sliding windows
            num_frames = joint_matrix.shape[0]
            for i in range(0, num_frames - window_size + 1, stride):
                window = joint_matrix[i:i+window_size]
                self.samples.append(torch.tensor(window, dtype=torch.float32))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


In [23]:
BENCH_TENSORS['bench 2 good lift data'].keys()

dict_keys(['bench 2 good landmark 0', 'bench 2 good landmark 1', 'bench 2 good landmark 2', 'bench 2 good landmark 3', 'bench 2 good landmark 4', 'bench 2 good landmark 5', 'bench 2 good landmark 6', 'bench 2 good landmark 7', 'bench 2 good landmark 8', 'bench 2 good landmark 9', 'bench 2 good landmark 10', 'bench 2 good landmark 11', 'bench 2 good landmark 12', 'bench 2 good landmark 13', 'bench 2 good landmark 14', 'bench 2 good landmark 15', 'bench 2 good landmark 16', 'bench 2 good landmark 17', 'bench 2 good landmark 18', 'bench 2 good landmark 19', 'bench 2 good landmark 20', 'bench 2 good landmark 21', 'bench 2 good landmark 22', 'bench 2 good landmark 23', 'bench 2 good landmark 24', 'bench 2 good landmark 25', 'bench 2 good landmark 26', 'bench 2 good landmark 27', 'bench 2 good landmark 28', 'bench 2 good landmark 29', 'bench 2 good landmark 30', 'bench 2 good landmark 31', 'bench 2 good landmark 32', 'bench 2 good left elbow', 'bench 2 good right elbow', 'bench 2 good left k

In [28]:
dataset = DeadliftDataset(BENCH_TENSORS['bench 2 good lift data'])
print(len(dataset))  # Number of sliding windows
print(dataset[0].shape)  # torch.Size([60, 8])


bench 2 good landmark 0 tensor([[0.5300, 0.5300, 0.5300, 0.5300, 0.5300, 0.5300, 0.5300, 0.5300, 0.5300,
         0.5400, 0.5400, 0.5400, 0.5400, 0.5500, 0.5500, 0.5500, 0.5400, 0.5300,
         0.5400, 0.5500, 0.5600, 0.5600, 0.5500, 0.5500, 0.5500, 0.5500, 0.5500,
         0.5500, 0.5500, 0.5500, 0.5500, 0.5600, 0.5600, 0.5600, 0.5600, 0.5600],
        [0.4200, 0.4200, 0.4200, 0.4200, 0.4200, 0.4200, 0.4200, 0.4200, 0.4200,
         0.4300, 0.4400, 0.4400, 0.4600, 0.4700, 0.4600, 0.4600, 0.4600, 0.4600,
         0.4400, 0.4500, 0.4600, 0.4600, 0.4500, 0.4500, 0.4500, 0.4600, 0.4600,
         0.4600, 0.4700, 0.4700, 0.4800, 0.4900, 0.4900, 0.4900, 0.5000, 0.5000],
        [0.3600, 0.3600, 0.3200, 0.3100, 0.2700, 0.3100, 0.2600, 0.2600, 0.2500,
         0.2700, 0.2800, 0.2800, 0.3000, 0.2900, 0.2600, 0.2600, 0.2700, 0.2400,
         0.3100, 0.3200, 0.3400, 0.3000, 0.2800, 0.2500, 0.2400, 0.1900, 0.2000,
         0.2300, 0.2200, 0.2200, 0.4500, 0.3500, 0.2500, 0.2800, 0.2900, 0.2900]],


TypeError: argument of type 'NoneType' is not iterable